## 线性回归的简洁实现

利用pytorch工具包来实现线性模型：

1. `torch.utils.data`模块提供有关数据处理的工具

2. `torch.nn`模块定义了大量神经网络的层

3. `troch.nn.init`提供了常用的优化算法

In [1]:
import torch
import numpy as np

# 生成数据集

num_example=1000
num_feature=2

true_w=[2,3.4]
true_b=4.2

# dataArr
dataArr=torch.tensor(np.random.normal(0,1,
    size=(num_example,num_feature,)),
    dtype=torch.float)
# labels
labels=true_w[0]*dataArr[:,0]+true_w[1]*dataArr[:,1]+true_b

# noise
labels+=torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float)

**读取数据**

使用`torch.utils.data`包，读取feature和labels为数据和标签的组合。

使用`data.DataLoader(dataset,batch_size,)`获取batch_size的数据。

In [2]:
import torch.utils.data as Data

# 一次读取batch_size个数据
batch_size=10

# 组合数据特征和标签
dataset=Data.TensorDataset(dataArr,labels)

# 随机读取批量数据
data_iter=Data.DataLoader(dataset,batch_size,shuffle=True)


In [3]:
# test
for X,y in data_iter:
    print(X,y)
    # 打印一次就可以了
    break 

tensor([[-0.5116,  0.4040],
        [ 0.6109,  1.3295],
        [-1.5260,  1.2462],
        [ 1.3314,  0.1926],
        [-0.2202, -0.0371],
        [ 0.7827,  0.8682],
        [-0.0138, -0.9928],
        [ 0.5717, -0.4736],
        [-0.2960, -2.2792],
        [ 0.3316, -0.1090]]) tensor([ 4.5505,  9.9601,  5.3854,  7.5171,  3.6353,  8.7261,  0.7932,  3.7302,
        -4.1464,  4.4881])


### 定义模型

`torch.nn`核心数据结构是`Module`，是个抽象类。

一个`nn.Module`实例应该包含以一些层以及前向传播方法，这些前向传播方法就是输出。

**方法1**

使用`nn.Module`定义模型

In [4]:
import torch.nn as nn
class LinearNet(nn.Module):
    def __init__(self,n_feature):
        super(LinearNet,self).__init__()
        self.linear=nn.Linear(n_feature,1)

    # 定义前向传播
    def forward(self,x):
        y=self.linear(x)
        return y

In [5]:
net1=LinearNet(num_feature)
print(net1)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


**方法2**

使用`nn.Sequential`定义模型

网络层次将会按照添加顺序，自动构造生成。

In [6]:
net2=nn.Sequential( nn.Linear(num_feature,1) 
    # other layer
    )
print(net2)

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)


**方法2另一种写法**

In [7]:
net3=nn.Sequential()

# 添加层
net3.add_module('linear',nn.Linear(num_feature,1))
# net.add(...)
print(net3)


Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


Note: 线性回顾输出层中的神经元和输入层中各个输入层完全连接，所以，线性回归的输出层又叫做全连接层。

**初始化模型参数**

使用`torch.nn.init`初始化模型参数。

如果使用net1,那么`net[0].weight`应该写为`net[0].linear.weight`，bias同。

使用`net[0].weight`访问，只有当net是`ModuleList`或`Sequential`时才可以。

In [8]:
from torch.nn import init

net=net2
# 初始化权重为：均值为0，方差为0.01的正态分布
init.normal_(net[0].weight,mean=0,std=0.01)
init.constant(net[0].bias,val=0)

Parameter containing:
tensor([0.], requires_grad=True)

###  定义损失函数

In [9]:
# 均方根误差
loss=nn.MSELoss()

### 定义优化算法

使用`torch.optim`模块，该模块提供了许多优化算法：SGD,Adam,RMSProp等。

In [10]:
import torch.optim as optim

# ls为学习率
optimizer=optim.SGD(net.parameters(),lr=0.03)

print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


**其它**

不同子网设置不同学习率：

```python
optimizer =optim.SGD(
# 如果对某个参数不指定学习率，就使用最外层的默认学习率
    [{'params': net.subnet1.parameters()}, # lr=0.03
    {'params': net.subnet2.parameters(), 'lr': 0.01}], 
    lr=0.03)
print(optimizer)
```

可以动态调整学习率：

```python
for param_group in optimizer.param_group:
    param_group['lr']*=0.1

```

### 训练模型


In [11]:
num_epoch=3

for epoch in range(1,num_epoch+1):
    for X,y in data_iter:
        out=net(X)
        l=loss(out,y.view(-1,1))
        # 梯度清零
        optimizer.zero_grad()
        # 反向传播
        l.backward()
        # optim.step函数，用来迭代模型参数，对批量中样本梯度求平均
        optimizer.step()
    print('epoch %d, loss:%f' % (epoch,l.item()))


epoch 1, loss:0.000312
epoch 2, loss:0.000046
epoch 3, loss:0.000035


### 参数比较

从`net`获得想要的层，然后就可以直接访问权重和偏差。

In [12]:
dense=net[0]

print(true_w,'\n',dense.weight)
print(true_b,'\n',dense.bias)

[2, 3.4] 
 Parameter containing:
tensor([[1.9994, 3.3999]], requires_grad=True)
4.2 
 Parameter containing:
tensor([4.2000], requires_grad=True)
